In [30]:
import pdfplumber as pp
import pypdf

## Set up file path for pdf to txt conversion

In [155]:
dsm5 = "data/clinical_data/DSM-5.pdf"

WHO = "data/clinical_data/WHO_Clinical_descriptions_and_diagnostic_requirements_ICD-11.pdf"

practice_guidline = "data/clinical_data/treatmentofpatientswithmajordepressivedisorder.pdf"

SCID = "data/SCID-5-CV.pdf"



pdf = pypdf.PdfReader(dsm5)

## check for bookmarks

In [156]:
# Check for bookmarks
if pdf.outline:
    print("✅ PDF has bookmarks!")
    print(f"Found {len(pdf.outline)} top-level bookmarks")
    
    # Show first few
    print("\nFirst 5 bookmarks:")
    for i, item in enumerate(pdf.outline[:5]):
        if hasattr(item, 'title'):
            print(f"  {i+1}. {item.title}")
else:
    print("❌ No bookmarks found")

✅ PDF has bookmarks!
Found 18 top-level bookmarks

First 5 bookmarks:
  1. Cover Page
  2. Title Page
  3. Copyright Page
  4. Contents
  5. DSM-5-TR Chairs and Review Groups


### feel free to skip, this just finds page numbers based on headings

In [89]:

found_depressive = False
stop = False

depressive_start = None
next_section = None
next_section_start = None

for item in pdf.outline:
    if stop:
        break
    
    # Check if it's a list (nested bookmarks)
    if isinstance(item, list):
        for subitem in item:
            if stop:
                break
                
            if hasattr(subitem, 'title'):
                # Found Depressive Disorders
                if 'Depressive' in subitem.title and not found_depressive:
                    depressive_start = pdf.get_destination_page_number(subitem) + 1
                    found_depressive = True
                    print(f"Depressive Disorders starts at page: {depressive_start}")
                
                # Found the next section after Depressive Disorders
                elif found_depressive and not stop:
                    next_section = subitem.title
                    next_section_start = pdf.get_destination_page_number(subitem) + 1
                    stop = True
                    print(f"Next section '{next_section}' starts at page: {next_section_start}")
    
    # Also check top-level items
    elif hasattr(item, 'title'):
        if 'Depressive' in item.title and not found_depressive:
            depressive_start = pdf.get_destination_page_number(item) + 1
            found_depressive = True
            print(f"Depressive Disorders starts at page: {depressive_start}")
        
        elif found_depressive and not stop:
            next_section = item.title
            next_section_start = pdf.get_destination_page_number(item) + 1
            stop = True
            print(f"Next section '{next_section}' starts at page: {next_section_start}")

if depressive_start and next_section_start:
    total_pages = next_section_start - depressive_start
    print(f"\nDepressive Disorders section: {total_pages} pages")
    print(f"Range: Pages {depressive_start} to {next_section_start - 1}")

# View page

In [ ]:

def view_page(page_num, pdfname):
    """View a specific page (1-indexed like in the PDF)"""
    with pp.open(pdfname) as pdf:
        # Convert to 0-indexed
        page = pdf.pages[page_num - 1]
        text = page.extract_text()
        
        print("=" * 60)
        print(f"PAGE {page_num}")
        print("=" * 60)
        print(text)
        print("=" * 60)

        return page_num, text

def get_page(page_num, pdfname):
    """View a specific page (1-indexed like in the PDF)"""
    with pp.open(pdfname) as pdf:
        # Convert to 0-indexed
        page = pdf.pages[page_num - 1]
        text = page.extract_text()

        return text

# Use it
page_num, text = view_page(455, dsm5)

#436-446
#455-507

PAGE 455
concentration or indecision, and suicidal thoughts should be assessed with
particular care in such cases. Definitions of major depressive episodes that
have been modified to include only these nonvegetative symptoms appear
to identify nearly the same individuals as do the full criteria.
Associated Features
Major depressive disorder is associated with high mortality, much of which
is accounted for by suicide; however, it is not the only cause. For example,
depressed individuals admitted to nursing homes have a markedly increased
likelihood of death in the first year. Individuals frequently present with
tearfulness, irritability, brooding, obsessive rumination, anxiety, phobias,
excessive worry over physical health, and complaints of pain (e.g.,
headaches; joint, abdominal, or other pains). In children, separation anxiety
may occur.
Although an extensive literature exists describing neuroanatomical,
neuroendocrinological, and neurophysiological correlates of major
depressive dis

## Create a chunk list

In [160]:
mandatory = []
WHO_chunk = []
APA_chunk = []
guide_1 = []
guide_2 = []
guide_3 = []
dsm5_chunk = []

## get list of pages

In [162]:
# input page numbers, output none
def append_middle_pages(page_numbers, book, my_list):
    for i in page_numbers:
        txt = get_page(i, book)
        my_list.append(txt)

page_numbers = list(range(455, 508))
append_middle_pages(page_numbers, dsm5, dsm5_chunk)

#436-446
#455-507

In [163]:
print(len(dsm5_chunk))

64


## Only use when extra info must be cut

In [ ]:
def cut_before_title(text, title):
    """Remove everything before the title and return text from title onwards"""
    # Find where the title appears (case-insensitive)
    title_lower = title.lower()
    text_lower = text.lower()
    
    # Find the position of the title
    title_pos = text_lower.find(title_lower)
    
    if title_pos != -1:
        # Return text from title onwards
        return text[title_pos:]
    else:
        # Title not found, return full text
        print(f"Warning: '{title}' not found in text")
        return text

page_num, current_text = view_page(262, WHO)
title = "Major Depressive Disorder"
start_text = cut_before_title(current_text, title)
mandatory.append(start_text)

print("=" * 60)
print(start_text)
print("=" * 60)

PAGE 447
Also, as noted earlier, the diagnosis of disruptive mood dysregulation
disorder should not be assigned if the symptoms occur only in an anxiety-
provoking context, when the routines of a child with autism spectrum
disorder or obsessive-compulsive disorder are disturbed, or in the context of
a major depressive episode.
183
Major Depressive Disorder
Diagnostic Criteria
A. Five (or more) of the following symptoms have been present
during the same 2-week period and represent a change from
previous functioning; at least one of the symptoms is either
(1) depressed mood or (2) loss of interest or pleasure.
Note: Do not include symptoms that are clearly attributable to
another medical condition.
1. Depressed mood most of the day, nearly every day, as
indicated by either subjective report (e.g., feels sad,
empty, hopeless) or observation made by others (e.g.,
appears tearful). (Note: In children and adolescents, can
be irritable mood.)
2. Markedly diminished interest or pleasure in all

# Get middle pages for mandatory chunk

In [76]:

# input page numbers, output none
def append_middle_pages(page_numbers):
    for i in page_numbers:
        txt = get_page(i, dsm5)
        mandatory.append(txt)

page_numbers = [448, 449, 450, 451, 452, 453, 454]
append_middle_pages(page_numbers)

In [79]:
print(len(mandatory))

9


## Get last page

In [67]:
num, txt = view_page(455, dsm5)

PAGE 455
concentration or indecision, and suicidal thoughts should be assessed with
particular care in such cases. Definitions of major depressive episodes that
have been modified to include only these nonvegetative symptoms appear
to identify nearly the same individuals as do the full criteria.
Associated Features
Major depressive disorder is associated with high mortality, much of which
is accounted for by suicide; however, it is not the only cause. For example,
depressed individuals admitted to nursing homes have a markedly increased
likelihood of death in the first year. Individuals frequently present with
tearfulness, irritability, brooding, obsessive rumination, anxiety, phobias,
excessive worry over physical health, and complaints of pain (e.g.,
headaches; joint, abdominal, or other pains). In children, separation anxiety
may occur.
Although an extensive literature exists describing neuroanatomical,
neuroendocrinological, and neurophysiological correlates of major
depressive dis

In [78]:
def cut_before_title(text, title):
    """Remove everything before the title and return text from title onwards"""
    # Find where the title appears (case-insensitive)
    title_lower = title.lower()
    text_lower = text.lower()
    
    # Find the position of the title
    title_pos = text_lower.find(title_lower)
    
    if title_pos != -1:
        # Return text from title onwards
        return text[:title_pos]
    else:
        # Title not found, return full text
        print(f"Warning: '{title}' not found in text")
        return text

end_text = get_page(455, dsm5)
title = "Associated Features"
end_text = cut_before_title(end_text, title)

print("=" * 60)
print(end_text)
print("=" * 60)

mandatory.append(end_text)

concentration or indecision, and suicidal thoughts should be assessed with
particular care in such cases. Definitions of major depressive episodes that
have been modified to include only these nonvegetative symptoms appear
to identify nearly the same individuals as do the full criteria.



In [132]:
for i in guide:
    print(i)

Practice Guideline for the Treatment of Patients With Major Depressive Disorder, Third Edition 77
ommended for the treatment of fibromyalgia in combina- ties so that patients do not receive prescriptions for the
tion with antidepressant medication (963, 964). same medications or have their doses for given medica-
In the context of rheumatoid arthritis, antidepressant tions adjusted by several different prescribing clinicians,
medications also appear to be effective in reducing pain as and to set up a mechanism and plan whereby all prescrib-
well as treating depressive symptoms (965–967). Evidence ing clinicians consistently keep one another informed
for psychosocial treatment is less consistent, with mind- about changes in their treatment plans and prescriptions.
fulness meditation and emotion regulation therapy being
associated with reduced pain and enhanced coping in in- 11. Obstructive uropathy
dividuals with rheumatoid arthritis and depression (968), Enlarged prostate size and othe

## Used when need to remove page number

In [164]:
def remove_page_numbers(text):
    """Remove lines that are just a single number (page numbers)"""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Strip whitespace
        stripped = line.strip()
        
        # Check if line is just a single number
        if stripped.isdigit():
            # Skip this line (it's a page number)
            continue
        else:
            # Keep this line
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

# Clean all items in your mandatory list
dsm5_chunk = [remove_page_numbers(text) for text in dsm5_chunk]

add section to remove certain phrases:  Copyright 2010, American Psychiatric Association. APA makes this practice guideline freely available to promote its dissemination and use; however, copyright protections are enforced in full. No part of this guideline may  
be reproduced except as permitted under Sections 107 and 108 of U.S. Copyright Act. For permission for reuse, visit APPI Permissions & Licensing Center at http://www.appi.org/CustomerService/Pages/Permissions.aspx.


In [ ]:
def remove_copyright_lines(text):
    """Remove lines that contain 'Copyright 2010'"""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Check if line contains "Copyright 2010"
        if "Copyright" in line:
            # Skip this line
            continue
        else:
            # Keep this line
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

guide_1 = [remove_copyright_lines(text) for text in guide_1]
guide_2 = [remove_copyright_lines(text) for text in guide_2]
guide_3 = [remove_copyright_lines(text) for text in guide_3]



In [166]:
for i in dsm5_chunk:
    print(i)

Depressive Disorders
Depressive disorders
include disruptive mood dysregulation
disorder, major depressive disorder (including major depressive episode),
persistent depressive disorder, premenstrual dysphoric disorder,
substance/medication-induced depressive disorder, depressive disorder due
to another medical condition, other specified depressive disorder, and
unspecified depressive disorder. The common feature of all of these
disorders is the presence of sad, empty, or irritable mood, accompanied by
related changes that significantly affect the individual’s capacity to function
(e.g., somatic and cognitive changes in major depressive disorder and
persistent depressive disorder). What differs among them are issues of
duration, timing, or presumed etiology.
In order to address concerns in the United States and, increasingly,
internationally about the potential for the overdiagnosis and treatment of
bipolar disorder in children, a new diagnosis, disruptive mood dysregulation
disorder, r

In [85]:
print(len(mandatory))

9


# Save to txt file

In [167]:

def save_as_txt(mylist, location):
    full_text = ''.join(mylist)
    with open(location, 'w', encoding='utf-8') as f:
        f.write(full_text)

save_as_txt(dsm5_chunk, 'data/clinical_data/to_chunk/dsm5')